In [3]:
import argparse
import json
import math
import os
from typing import List, Tuple, Dict

from tqdm import tqdm

from nuscenes.nuscenes import NuScenes


EARTH_RADIUS_METERS = 6.378137e6
REFERENCE_COORDINATES = {
    "boston-seaport": [42.336849169438615, -71.05785369873047],
    "singapore-onenorth": [1.2882100868743724, 103.78475189208984],
    "singapore-hollandvillage": [1.2993652317780957, 103.78217697143555],
    "singapore-queenstown": [1.2782562240223188, 103.76741409301758],
}

dataroot = '/disk/ml/datasets/nuScenes/'

json_tokens = '/disk/ml/own_datasets/CODA/nuscenes_indices.json'

poses_folder = '/disk/vanishing_data/ju878/nusc_kitti_test/train/oxts/'

if not os.path.exists(poses_folder):
        os.makedirs(poses_folder)

In [ ]:
nusc = NuScenes(dataroot=dataroot, version='v1.0-trainval', verbose=False)

In [3]:
def get_poses(nusc: NuScenes, scene_token: str) -> List[dict]:
    """
    Return all ego poses for the current scene.
    :param nusc: The NuScenes instance to load the ego poses from.
    :param scene_token: The token of the scene.
    :return: A list of the ego pose dicts.
    """
    pose_list = []
    scene_rec = nusc.get('scene', scene_token)
    sample_rec = nusc.get('sample', scene_rec['first_sample_token'])
    sd_rec = nusc.get('sample_data', sample_rec['data']['LIDAR_TOP'])
    
    
    ego_pose = nusc.get('ego_pose', sd_rec['token'])
    #if sd_rec['token'] in lidar_top_tokens:
    pose_list.append(ego_pose)

    while sd_rec['next'] != '':
        sd_rec = nusc.get('sample_data', sd_rec['next'])
        ego_pose = nusc.get('ego_pose', sd_rec['token'])
        #if sd_rec['token'] in lidar_top_tokens:
        pose_list.append(ego_pose)

    return pose_list

In [4]:
def get_coordinate(ref_lat: float, ref_lon: float, bearing: float, dist: float) -> Tuple[float, float]:
    """
    Using a reference coordinate, extract the coordinates of another point in space given its distance and bearing
    to the reference coordinate. For reference, please see: https://www.movable-type.co.uk/scripts/latlong.html.
    :param ref_lat: Latitude of the reference coordinate in degrees, ie: 42.3368.
    :param ref_lon: Longitude of the reference coordinate in degrees, ie: 71.0578.
    :param bearing: The clockwise angle in radians between target point, reference point and the axis pointing north.
    :param dist: The distance in meters from the reference point to the target point.
    :return: A tuple of lat and lon.
    """
    lat, lon = math.radians(ref_lat), math.radians(ref_lon)
    angular_distance = dist / EARTH_RADIUS_METERS
    
    target_lat = math.asin(
        math.sin(lat) * math.cos(angular_distance) + 
        math.cos(lat) * math.sin(angular_distance) * math.cos(bearing)
    )
    target_lon = lon + math.atan2(
        math.sin(bearing) * math.sin(angular_distance) * math.cos(lat),
        math.cos(angular_distance) - math.sin(lat) * math.sin(target_lat)
    )
    return math.degrees(target_lat), math.degrees(target_lon)

In [7]:
def derive_latlon(location: str, poses: List[Dict[str, float]], scene_name) -> List[Dict[str, float]]:
    """
    For each pose value, extract its respective lat/lon coordinate and timestamp.
    
    This makes the following two assumptions in order to work:
        1. The reference coordinate for each map is in the south-western corner.
        2. The origin of the global poses is also in the south-western corner (and identical to 1).

    :param location: The name of the map the poses correspond to, ie: 'boston-seaport'.
    :param poses: All nuScenes egopose dictionaries of a scene.
    :return: A list of dicts (lat/lon coordinates and timestamps) for each pose.
    """
    assert location in REFERENCE_COORDINATES.keys(), \
        f'Error: The given location: {location}, has no available reference.'
    
    coordinates = []
    reference_lat, reference_lon = REFERENCE_COORDINATES[location]
    for p in poses:
        ts = p['timestamp']
        pose_token = p['token']
        #file_name = 'error'
        #for index, lidar_top_token in enumerate(lidar_top_tokens):
        #    ego_pose = nusc.get('ego_pose', lidar_top_tokens[index])
        #    if pose_token == ego_pose['token']:
        #        file_name = file_names[index]
        x, y, z = p['translation']
        bearing = math.atan(x / y)
        distance = math.sqrt(x**2 + y**2)
        lat, lon = get_coordinate(reference_lat, reference_lon, bearing, distance)
        
        w, r1, r2, r3 = p['rotation']
        roll = math.atan2(2 * (w * r1 + r2 * r3), 1 - 2 * (math.pow(r1, 2) + math.pow(r2, 2)))
        pitch = math.asin(2 * (w * r2 - r3 * r1))
        yaw = math.atan2(2 * (w * r3 + r1 * r2), 1 - 2 * (math.pow(r2, 2) + math.pow(r3, 2)))
        #coordinates.append(lat + ' ' + lon + ' ' + z + ' ' + roll + ' ' + pitch + ' ' + yaw)
        if not os.path.exists(poses_folder + '/' + scene_name +'/'):
            os.makedirs(poses_folder + '/' + scene_name +'/')
        with open(poses_folder + '/' + scene_name +'/'+ pose_token + '.txt', 'w') as f:
            f.write(f'{lat} {lon} {z} {roll} {pitch} {yaw}')
    #return coordinates

In [8]:


#with open(json_tokens, 'r') as f:
#    nuscenes_tokens = json.load(f)
#       
#file_names = []
#lidar_top_tokens = []
#
#for token in nuscenes_tokens:
#    file_names.append(token.split('.')[0])
#    my_sample = nusc.get('sample', nuscenes_tokens[token])
#    lidar_top_data = nusc.get('sample_data', my_sample['data']['LIDAR_TOP'])
#    lidar_top_tokens.append(lidar_top_data['token'])

coordinates_per_location = {}
print(f'Extracting coordinates...')
for scene in nusc.scene:
    # Retrieve nuScenes poses.
    scene_name = scene['name']
    scene_token = scene['token']
    location = nusc.get('log', scene['log_token'])['location']  # Needed to extract the reference coordinate.
    poses = get_poses(nusc, scene_token)  # For each pose, we will extract the corresponding coordinate.
    
    # Compute and store coordinates.
    derive_latlon(location, poses, scene_name)

Extracting coordinates...


: 